In [ ]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Parcial 2016 2do Cuatrimestre, Ejercicio 2 

En este ejercicio queremos programar un sistema que recomiende
textos a usuarios en base a sus gustos sobre ciertos términos (palabras).

Se cuenta con un RDD de textos de la forma (docId, texto) donde texto
es un string de longitud variable. 

Además contamos con un RDD que
indica qué términos le gustan o no a cada usuario de la forma (userId,
término, score) por ejemplo (23, “calesita”, -2). 

Se pide programar en Spark un programa que calcule el score total de cada documento para cada usuario generando un RDD de la forma (userId, docId, score) en donde el score es simplemente la suma de los scores del usuario para
los términos que aparecen en el documento. 

Puede haber términos en los documentos para los cuales no exista score de algunos usuarios, en
estos casos simplemente los consideramos neutros (score=0)

In [ ]:
documents = [
    (1, 'pablo honey'),
    (2, 'the bends'),
    (3, 'ok computer'),
    (4, 'kid a'),
    (5, 'amnesiac'),
    (6, 'hail to the thief'),
    (7, 'in rainbows'),
    (8, 'the king of limbs'),
    (9, 'a moon shaped pool')
]

scores = [
    ('thom', 'pablo', 1),
    ('thom', 'honey', 1),
    ('martin', 'pablo', -1),
    ('martin', 'honey', -1),
    ('martin', 'ok', 30),
    ('martin', 'computer', 30)
]

In [ ]:
documentsRDD = sc.parallelize(documents)
scoresRDD = sc.parallelize(scores)

In [ ]:
# generamos algo del tipo (word, docId) donde word sera nuestra key
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()]).collect()

In [ ]:
# llevamos scores a una representacion que nos permita unir la informacion
scoresRDDForJoin = scoresRDD.map(lambda a: (a[1],(a[0],a[2])))
scoresRDDForJoin.collect()

In [ ]:
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .collect()

In [ ]:
# para poder realizar una acumulacion debemos llevar la informacion a la representacion (docId, userId, score) 
# y luego realizar una acumulacion de scores
# ignorando el score

# notese que el K es docId, userId y el V es el score que aporta el termino

documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .collect()

In [ ]:
# acumulamos haciendo reduce
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .reduceByKey(lambda a,b: a + b)\
    .collect()

In [ ]:
# llevamos a la representacion pedida
documentsRDD.flatMap(lambda a: [(word, a[0]) for word in a[1].split()])\
    .join(scoresRDDForJoin)\
    .map(lambda a: ((a[1][0], a[1][1][0]), a[1][1][1]))\
    .reduceByKey(lambda a,b: a + b)\
    .map(lambda a: (a[0][0], a[0][1], a[1]))\
    .collect()

# Parcial 2016 2do Cuatrimestre, Ejercicio 1

Contamos con un cluster que tiene 4 computadoras. 

Queremos aprovechar el paralelismo del cluster para calcular los números primos entre 2 y 20.000.000. Para esto usaremos el conocido algoritmo de la
criba de Eratóstenes. Por ejemplo si empezamos con una lista de tipo (2,3,4,5,6,7,8...) en un primer paso eliminamos todos los que son
mayores a 2 y divisibles por 2 y nos queda (2,3,5,7,9,11,13…) luego
eliminamos todos los mayores a 3 divisibles por 3 y nos queda
(2,3,5,7,11,13….etc) luego todos los divisibles por 5 y así
sucesivamente. 

El resultado final es una lista de números que son
primos. 

Programar este programa usando PySpark

In [2]:
# criterio
# En primer lugar hay que crear un RDD con todos los números y hacer un parallelize con 4 particiones. 
# Luego la resolución pasa por un ciclo en el cual en cada paso del ciclo se obtenga 
# como pivote el mínimo número del RDD que sea mayor al número
# usado como pivote anteriormente, luego simplemente hay que 
# filtrar el RDD eliminando los números que son múltiplos del pivote

In [1]:
sc

In [5]:
def is_multiple(pivot, value):
        if (value == pivot):
            return True
    
        if (value % pivot) > 0:
            return True
        else:
            return False
        

In [6]:
print is_multiple(3, 6)

False


In [7]:
numbers.filter(lambda a: not is_multiple(2, a)).collect()

[4, 6, 8, 10, 12, 14, 16, 18, 20]

In [8]:
numbers = sc.parallelize(range(2,21), 4)
pivot = 2

numbers.collect()

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [9]:
while ((pivot*pivot) < 20):
    print 'current_pivot: ' + str(pivot)
    print numbers.filter(lambda a: is_multiple(2, a)).collect()
    new_pivot = numbers.filter(lambda a: a > pivot).reduce(lambda a, b: a if a < b else b)
    pivot = new_pivot

numbers.collect()

current_pivot: 2
[2, 3, 5, 7, 9, 11, 13, 15, 17, 19]
current_pivot: 3
[2, 3, 5, 7, 9, 11, 13, 15, 17, 19]
current_pivot: 4
[2, 3, 5, 7, 9, 11, 13, 15, 17, 19]


[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [71]:
numbers.take(5)

[2, 3, 4, 5, 6]